## **Training MobileNetSSD Object Detection on a Custom Dataset**

We'll be training a MobileNetSSDv2 (single shot detector). This specific model is a one-short learner, meaning each image only passes through the network once to make a prediction, which allows the architecture to be very performant for mobile hardware.

The model arechitecture is one of many available via TensorFlow's [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

As a note, this notebook presumes TensorFlow 1.5 as TensorFlow 2.0 has yet to fully support the object detection API.

### **Training**

Google Colab provides free GPU resources. Click "Runtime" → "Change runtime type" → Hardware Accelerator dropdown to "GPU."

Colab does have memory limitations, and notebooks must be open in your browser to run. Sessions automatically clear themselves after 12 hours.

### **Inference**

We'll run inference directly in this notebook, and on three test images contained in the "test" folder from our GitHub repo. 

When adapting to your own dataset, you'll need to add test images to the `test` folder located at `tensorflow-object-detection/test`.







## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SSD_mobilenet

Mounted at /content/drive
/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 5.2 MB/s 
     |████████████████████████████████| 235 kB 56.4 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 48 kB 5.7 MB/s 
     |████████████████████████████████| 4.1 MB 63.7 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=f2dd97b36a9cc44dae61241e9b25c5bded7c81f8ad87f1af89eb76cc3dd0510f
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 200000 

# Number of evaluation steps.
num_eval_steps = 50000


MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 32
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    },    
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
import os

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

# !git clone {repo_url}
# %cd {repo_dir_path}
# !git pull

## Install required packages

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/drive/MyDrive/SSD_mobilenet/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/SSD_mobilenet/models/research/:/content/drive/MyDrive/SSD_mobilenet/models/research/slim/'

!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

!python object_detection/builders/model_builder_test.py

/content/drive/MyDrive/SSD_mobilenet
fatal: destination path 'models' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/SSD_mobilenet/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.4.0 which is inc

Found existing installation: pycocotools 2.0.4
Uninstalling pycocotools-2.0.4:
  Successfully uninstalled pycocotools-2.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pycocotools-2.0.4.tar.gz (106 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=265192 sha256=968c62dc071a4492934c3e99d3aa7d85e5ec7e7ff7498c08dfc378a54435eaf3
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.9.2 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
tf-models-official 2.

## Prepare `tfrecord` files

Roboflow automatically creates our TFRecord and label_map files that we need!

**Generating your own TFRecords the only step you need to change for your own custom dataset.**

Because we need one TFRecord file for our training data, and one TFRecord file for our test data, we'll create two separate datasets in Roboflow and generate one set of TFRecords for each.

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data

/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data


In [ ]:
!curl -L "https://app.roboflow.com/ds/78cv5m7bVB?key=IKe4ZS7rFl" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   2723      0 --:--:-- --:--:-- --:--:--  2723
100  203M  100  203M    0     0  64.4M      0  0:00:03  0:00:03 --:--:-- 82.5M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/vehicles.tfrecord  
 extracting: test/vehicles_label_map.pbtxt  
   creating: train/
 extracting: train/vehicles.tfrecord  
 extracting: train/vehicles_label_map.pbtxt  


In [ ]:
# training set
%ls train

vehicles_label_map.pbtxt  vehicles.tfrecord


In [ ]:
# test set
%ls test

vehicles_label_map.pbtxt  vehicles.tfrecord


In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles.tfrecord'
train_record_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/train/vehicles.tfrecord'
label_map_pbtxt_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/train/vehicles_label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model
total 135M
-rw------- 1 root root   77 Mar 30  2018 checkpoint
-rw------- 1 root root  67M Mar 30  2018 frozen_inference_graph.pb
-rw------- 1 root root  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw------- 1 root root  15K Mar 30  2018 model.ckpt.index
-rw------- 1 root root 3.4M Mar 30  2018 model.ckpt.meta
-rw------- 1 root root 4.2K Mar 30  2018 pipeline.config
drwx------ 3 root root 4.0K Mar 30  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 11
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect

{'/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'}

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)

## Train the model

### 200k steps 32 batch size Started at 10.48am stopped at 7am


In [ ]:
!python /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

Streaming output truncated to the last 5000 lines.
2022-06-22 16:28:55.863708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 16:28:55.864300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 16:28:55.864801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1325] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 15224 MB memory) -> physical GPU (device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0)
INFO:tensorflow:Restoring parameters from training/model.ckpt-102056
I0622 16:28:55.868102 140645494118272 saver.py:1284] Restoring parameters from training/model.ckpt-102056
INFO:tensorflow:Running local_init_op.
I0622 16:

In [ ]:
!ls 

eval		  README.dataset.txt			   test
fine_tuned_model  README.roboflow.txt			   train
models		  tensorflow-object-detection-faster-rcnn  training


In [ ]:
!tensorboard --logdir=.

TensorBoard 1.15.0 at http://b560081212f5:6007/ (Press CTRL+C to quit)
^C


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
# !python /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/export_inference_graph.py \
#     --input_type=image_tensor \
#     --pipeline_config_path={pipeline_fname} \
#     --output_directory={output_directory} \
#     --trained_checkpoint_prefix={last_model_path}

training/model.ckpt-148285


In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

-rw------- 1 root root 20M Jun 22 23:19 /content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/frozen_inference_graph.pb


### Option1 : upload the `.pb` file to your Google Drive
Then download it from your Google Drive to local file system.

During this step, you will be prompted to enter the token.

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fname = os.path.basename(pb_fname)
# Create & upload a text file.
uploaded = drive.CreateFile({'title': fname})
uploaded.SetContentFile(pb_fname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1WRg5dXRH5-b5UZ3cK-nbhLktQlrPRLgV


### Option2 :  Download the `.pb` file directly to your local file system
This method may not be stable when downloading large files like the model `.pb` file. Try **option 1** instead if not working.

In [ ]:
from google.colab import files
files.download(pb_fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### OPTIONAL: Download the `label_map.pbtxt` file

In [ ]:
from google.colab import files
files.download(label_map_pbtxt_fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### OPTIONAL: Download the modified pipline file
If you plan to use OpenVINO toolkit to convert the `.pb` file to inference faster on Intel's hardware (CPU/GPU, Movidius, etc.)

In [ ]:
files.download(pipeline_fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !tar cfz fine_tuned_model.tar.gz fine_tuned_model
# from google.colab import files
# files.download('fine_tuned_model.tar.gz')

## Run inference test
Test with images in repository `tensorflow-object-detection/test` directory.

**To test with your own images, you need to place your images inside the `test` directory in this Colab notebook!** More on this below.

In [ ]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  repo_dir_path + "/data/test/"
# PATH_TO_TEST_IMAGES_DIR = 
# sample_img = 'https://storage.googleapis.com/roboflow-platform-transforms/Ly2DeBzbwsemGd2ReHk4BFxy8683/cf5ed147e4f2675fbabbc9b0db750ecf/transformed.jpg'
# import urllib.request
# urllib.request.urlretrieve(sample_img, 
#                            PATH_TO_TEST_IMAGES_DIR + "cell.jpg")


assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

['/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles_label_map.pbtxt', '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles.tfrecord']


In [ ]:
print(TEST_IMAGE_PATHS)

['/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles_label_map.pbtxt', '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles.tfrecord']


In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection


In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  try:
    image = Image.open(image_path)
    print(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    plt.show()
  except Exception:
    pass

In [ ]:
### Adding your own images to tensorflow-object-detection/data
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

In [ ]:
# navigate to correct folder
%cd /content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/

# call function to upload
upload_files()

/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test


Saving bicycle--4-_jpg.rf.ca5a5cc63917d22af07958b1f376bb4a.jpg to bicycle--4-_jpg.rf.ca5a5cc63917d22af07958b1f376bb4a.jpg
Saving bicycle--32-_jpg.rf.23faa3cd2ca04f62f8addc60e10845ce.jpg to bicycle--32-_jpg.rf.23faa3cd2ca04f62f8addc60e10845ce.jpg
Saving bicycle--39-_jpg.rf.b242e6f37e2e68c0b613f3c5803cf62d.jpg to bicycle--39-_jpg.rf.b242e6f37e2e68c0b613f3c5803cf62d.jpg
Saving bicycle--42-_jpg.rf.6c478198af0d468a83ecb765141a3357.jpg to bicycle--42-_jpg.rf.6c478198af0d468a83ecb765141a3357.jpg
Saving bicycle--58-_jpg.rf.2bbfd017acccc2588b5fa59ae32104e1.jpg to bicycle--58-_jpg.rf.2bbfd017acccc2588b5fa59ae32104e1.jpg
Saving bicycle--74-_jpg.rf.abf5d302c3669cf593a9785bc6cc486d.jpg to bicycle--74-_jpg.rf.abf5d302c3669cf593a9785bc6cc486d.jpg
Saving bicycle--81-_jpg.rf.fb81e5707556628f5cdcc550071f7dd2.jpg to bicycle--81-_jpg.rf.fb81e5707556628f5cdcc550071f7dd2.jpg
Saving bicycle--85-_jpg.rf.0bbd32fe5e6b78f91e2c94ee43f6ca7c.jpg to bicycle--85-_jpg.rf.0bbd32fe5e6b78f91e2c94ee43f6ca7c.jpg
Saving bic

['bicycle--4-_jpg.rf.ca5a5cc63917d22af07958b1f376bb4a.jpg',
 'bicycle--32-_jpg.rf.23faa3cd2ca04f62f8addc60e10845ce.jpg',
 'bicycle--39-_jpg.rf.b242e6f37e2e68c0b613f3c5803cf62d.jpg',
 'bicycle--42-_jpg.rf.6c478198af0d468a83ecb765141a3357.jpg',
 'bicycle--58-_jpg.rf.2bbfd017acccc2588b5fa59ae32104e1.jpg',
 'bicycle--74-_jpg.rf.abf5d302c3669cf593a9785bc6cc486d.jpg',
 'bicycle--81-_jpg.rf.fb81e5707556628f5cdcc550071f7dd2.jpg',
 'bicycle--85-_jpg.rf.0bbd32fe5e6b78f91e2c94ee43f6ca7c.jpg',
 'bicycle--91-_jpg.rf.8624f012604f2530c16c6cbf444ccb92.jpg',
 'bicycle--102-_jpg.rf.31765b31b8db4aad19c80fe71a0793f1.jpg',
 'bicycle--124-_jpg.rf.80b36ce517985db2972198ad98bcfc8f.jpg',
 'bicycle--130-_jpg.rf.7b349c30ff51e7693e180b5ee00b45dd.jpg',
 'bicycle--134-_jpg.rf.ecc798a1b148d2985b3797fed8100610.jpg',
 'bicycle--136-_jpg.rf.53feba5aaa17b8f031349066b492337b.jpg',
 'bicycle--141-_jpg.rf.edddd639e71b0f52ee9778983e822deb.jpg',
 'bicycle--144-_jpg.rf.b7bae18c68eb3b181d7a5dbfafc28fe5.jpg',
 'bicycle--150-_jp

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection

/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection


In [ ]:
checkpoint='/content/drive/MyDrive/SSD_mobilenet/ds1/training1/'
eval='/content/drive/MyDrive/SSD_mobilenet/ds1/fine_tuned_model/saved_model'
pipeline_config='/content/drive/MyDrive/SSD_mobilenet/ds1/fine_tuned_model/pipeline.config'

In [ ]:
!ls /content/drive/MyDrive/SSD_mobilenet/ds1/fine_tuned_model/

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [ ]:
!python model_main.py --logtostderr --checkpoint_dir=checkpoint --pipeline_config_path='/content/drive/MyDrive/SSD_mobilenet/ds1/fine_tuned_model/pipeline.config' --model_dir=eval --run_once=True
        # --pipeline_config_path=pipeline_config.pbtxt


Using TensorFlow backend.
W0623 08:04:14.334034 140392617781120 model_lib.py:839] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0623 08:04:14.334292 140392617781120 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0623 08:04:14.334420 140392617781120 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0623 08:04:14.334521 140392617781120 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0623 08:04:14.334613 140392617781120 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0623 08:04:14.334728 140392617781120 model_lib.py:855] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: 

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet

/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!python export_tflite_ssd_graph.py --pipeline_config_path='/content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/pipeline.config' --trained_checkpoint_prefix='/content/drive/MyDrive/SSD_mobilenet/training/model.ckpt-148285' --output_directory=tflite --add_postprocessing_op=true

Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0623 08:29:41.282077 139966975330176 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0623 08:29:43.997847 139966975330176 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0623 08:29:44.036519 139966975330176 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0623 08:29:44.073756 139966975330176 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:dep

## Dataset 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SSD_mobilenet

Mounted at /content/drive
/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 4.3 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 235 kB 68.5 MB/s 
     |████████████████████████████████| 48 kB 6.2 MB/s 
     |████████████████████████████████| 4.1 MB 78.5 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=f28a9b10b518b09f977fb37a25488621202586d26c9db6cee8f2c26024979af4
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 150000 

# Number of evaluation steps.
num_eval_steps = 50000


MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 32
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 16
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    },    
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
import os

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

# !git clone {repo_url}
# %cd {repo_dir_path}
# !git pull

## Install required packages

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/drive/MyDrive/SSD_mobilenet/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/SSD_mobilenet/models/research/:/content/drive/MyDrive/SSD_mobilenet/models/research/slim/'

!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

!python object_detection/builders/model_builder_test.py

/content/drive/MyDrive/SSD_mobilenet
fatal: destination path 'models' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 5.1 MB/s 
Selecting previously unselected package python-bs4.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected p

Found existing installation: pycocotools 2.0.4
Uninstalling pycocotools-2.0.4:
  Successfully uninstalled pycocotools-2.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=265203 sha256=e846e8d174866777c735df0af43ea6ca1453ebeed1b0cbc546311c4b356137a1
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools
Using TensorFlow backend.


## Prepare `tfrecord` files

Roboflow automatically creates our TFRecord and label_map files that we need!

**Generating your own TFRecords the only step you need to change for your own custom dataset.**

Because we need one TFRecord file for our training data, and one TFRecord file for our test data, we'll create two separate datasets in Roboflow and generate one set of TFRecords for each.

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data

/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data


In [ ]:
!curl -L "https://app.roboflow.com/ds/eIxW7kw9vB?key=4aZSWYitF4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   2279      0 --:--:-- --:--:-- --:--:--  2279
100 54.3M  100 54.3M    0     0  46.6M      0  0:00:01  0:00:01 --:--:-- 84.2M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/vehicles.tfrecord  
 extracting: test/vehicles_label_map.pbtxt  
   creating: train/
 extracting: train/vehicles.tfrecord  
 extracting: train/vehicles_label_map.pbtxt  


In [ ]:
# training set
%ls train

vehicles_label_map.pbtxt  vehicles.tfrecord


In [ ]:
# test set
%ls test

bicycle--102-_jpg.rf.31765b31b8db4aad19c80fe71a0793f1.jpg
bicycle--124-_jpg.rf.80b36ce517985db2972198ad98bcfc8f.jpg
bicycle--130-_jpg.rf.7b349c30ff51e7693e180b5ee00b45dd.jpg
bicycle--134-_jpg.rf.ecc798a1b148d2985b3797fed8100610.jpg
bicycle--136-_jpg.rf.53feba5aaa17b8f031349066b492337b.jpg
bicycle--141-_jpg.rf.edddd639e71b0f52ee9778983e822deb.jpg
bicycle--144-_jpg.rf.b7bae18c68eb3b181d7a5dbfafc28fe5.jpg
bicycle--150-_jpg.rf.0bda4bc75e421780de2f3e5049b82223.jpg
bicycle--178-_jpg.rf.3ebd0637ea7990a38ae52e8563955609.jpg
bicycle--181-_jpg.rf.b53e392694df3e6ef522c882b86a8849.jpg
bicycle--223-_jpg.rf.48d888accb458b3212069d40243913b1.jpg
bicycle--229-_jpg.rf.0c94ff8e4f0b0b55db3bfd841684018b.jpg
bicycle--249-_jpg.rf.c6c344b272f43b781fcafa08bb4049bc.jpg
bicycle--266-_jpg.rf.34e4645a4d48bb533193f757d93012ba.jpg
bicycle--267-_jpg.rf.2220701d8f671015bfcffc21dd474323.jpg
bicycle--276-_jpg.rf.1c01ac40ad05fdb5dc74d0600637967c.jpg
bicycle--287-_jpg.rf.5ea0c6f43e35d0fa89d8b227f76d8713.jpg
bicycle--297-_

In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles.tfrecord'
train_record_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/train/vehicles.tfrecord'
label_map_pbtxt_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/train/vehicles_label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model
total 111M
-rw------- 1 root root   77 Feb  1  2018 checkpoint
-rw------- 1 root root  55M Feb  1  2018 frozen_inference_graph.pb
-rw------- 1 root root  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw------- 1 root root  16K Feb  1  2018 model.ckpt.index
-rw------- 1 root root 5.5M Feb  1  2018 model.ckpt.meta
-rw------- 1 root root 3.2K Feb  1  2018 pipeline.config
drwx------ 3 root root 4.0K Feb  1  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 11
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        tr

{'/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'}

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Train the model

### 150k steps 32 batch size Started at 19pm stopped at 7am


In [ ]:
!python /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:Scale of 0 disables regularizer.
I0719 17:25:55.953133 140676340299648 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Done calling model_fn.
I0719 17:25:58.734120 140676340299648 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-19T17:25:58Z
I0719 17:25:58.754750 140676340299648 evaluation.py:255] Starting evaluation at 2022-07-19T17:25:58Z
INFO:tensorflow:Graph was finalized.
I0719 17:25:59.322273 140676340299648 monitored_session.py:240] Graph was finalized.
2022-07-19 17:25:59.323088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 17:25:59.323606: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.

In [ ]:
!ls 

eval		  README.dataset.txt			   test
fine_tuned_model  README.roboflow.txt			   train
models		  tensorflow-object-detection-faster-rcnn  training


In [ ]:
!tensorboard --logdir=.

TensorBoard 1.15.0 at http://b560081212f5:6007/ (Press CTRL+C to quit)
^C


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

training/model.ckpt-100000
Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0706 00:48:32.262672 140157919401856 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0706 00:48:34.816002 140157919401856 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0706 00:48:34.862345 140157919401856 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0706 00:48:34.907845 140157919401856 convolutional_box_predictor.py:155] depth of additional conv before box predi

In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

-rw------- 1 root root 19M Jul  6 00:48 /content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/frozen_inference_graph.pb


In [ ]:
!python model_main.py --logtostderr --checkpoint_dir=checkpoint --pipeline_config_path='/content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/pipeline.config' --model_dir='/content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/frozen_inference_graph.pb' \
 --run_once=True
        # --pipeline_config_path=pipeline_config.pbtxt


python3: can't open file 'model_main.py': [Errno 2] No such file or directory


## Faster- RCNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SSD_mobilenet

Mounted at /content/drive
/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 8.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 48 kB 4.8 MB/s 
     |████████████████████████████████| 4.1 MB 42.6 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=96bbef36883abd7abb6a937d9c0fa1ca1a294e02c5db18d70486e3fcc9c33f8f
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 100000 

# Number of evaluation steps.
num_eval_steps = 50000


MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 32
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 32
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 32
    },    
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [ ]:
# use TF 1.x for Object Detection APIs as they are not ported to TF 2.0 yet
%tensorflow_version 1.x

## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
import os

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

# !git clone {repo_url}
# %cd {repo_dir_path}
# !git pull

## Install required packages

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/drive/MyDrive/SSD_mobilenet/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/SSD_mobilenet/models/research/:/content/drive/MyDrive/SSD_mobilenet/models/research/slim/'

!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

!python object_detection/builders/model_builder_test.py

/content/drive/MyDrive/SSD_mobilenet
fatal: destination path 'models' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 7.9 MB/s 
Selecting previously unselected package python-bs4.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected p

Found existing installation: pycocotools 2.0.4
Uninstalling pycocotools-2.0.4:
  Successfully uninstalled pycocotools-2.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 10.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=265206 sha256=5bd043f0a4b321109c256ba1a812d8569103081a103ae3cefb02652444510c20
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools
Using TensorFlow backend.


## Prepare `tfrecord` files

Roboflow automatically creates our TFRecord and label_map files that we need!

**Generating your own TFRecords the only step you need to change for your own custom dataset.**

Because we need one TFRecord file for our training data, and one TFRecord file for our test data, we'll create two separate datasets in Roboflow and generate one set of TFRecords for each.

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data

/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data


In [ ]:
!curl -L "https://app.roboflow.com/ds/eIxW7kw9vB?key=4aZSWYitF4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1746      0 --:--:-- --:--:-- --:--:--  1743
100 54.3M  100 54.3M    0     0  34.5M      0  0:00:01  0:00:01 --:--:-- 73.2M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/vehicles.tfrecord  
 extracting: test/vehicles_label_map.pbtxt  
   creating: train/
 extracting: train/vehicles.tfrecord  
 extracting: train/vehicles_label_map.pbtxt  


In [ ]:
# training set
%ls train

vehicles_label_map.pbtxt  vehicles.tfrecord


In [ ]:
# test set
%ls test

vehicles_label_map.pbtxt  vehicles.tfrecord


In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/test/vehicles.tfrecord'
train_record_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/train/vehicles.tfrecord'
label_map_pbtxt_fname = '/content/drive/MyDrive/SSD_mobilenet/tensorflow-object-detection-faster-rcnn/data/train/vehicles_label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/drive/MyDrive/SSD_mobilenet

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/drive/MyDrive/SSD_mobilenet


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model
total 111M
-rw------- 1 root root   77 Feb  1  2018 checkpoint
-rw------- 1 root root  55M Feb  1  2018 frozen_inference_graph.pb
-rw------- 1 root root  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw------- 1 root root  16K Feb  1  2018 model.ckpt.index
-rw------- 1 root root 5.5M Feb  1  2018 model.ckpt.meta
-rw------- 1 root root 3.2K Feb  1  2018 pipeline.config
drwx------ 3 root root 4.0K Feb  1  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/drive/MyDrive/SSD_mobilenet/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 4
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        tru

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Train the model

### 100k steps 32 batch size Started at 9am stopped at 


In [ ]:
!python /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

Streaming output truncated to the last 5000 lines.
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2022-07-06 17:16:24.911449: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-06 17:16:24.911484: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-06 17:16:24.911513: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-06 17:16:24.911541: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-06 17:16:24.911575: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-06 17:16:24.911602: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library l

In [ ]:
!ls 

eval		  README.dataset.txt			   test
fine_tuned_model  README.roboflow.txt			   train
models		  tensorflow-object-detection-faster-rcnn  training


In [ ]:
!tensorboard --logdir=.

TensorBoard 1.15.0 at http://b560081212f5:6007/ (Press CTRL+C to quit)
^C


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/drive/MyDrive/SSD_mobilenet/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

training/model.ckpt-100000
Using TensorFlow backend.
Instructions for updating:
Please use `layer.__call__` method instead.
W0706 00:48:32.262672 140157919401856 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0706 00:48:34.816002 140157919401856 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0706 00:48:34.862345 140157919401856 convolutional_box_predictor.py:155] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0706 00:48:34.907845 140157919401856 convolutional_box_predictor.py:155] depth of additional conv before box predi

In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

-rw------- 1 root root 19M Jul  6 00:48 /content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/frozen_inference_graph.pb


In [ ]:
!python model_main.py --logtostderr --checkpoint_dir=checkpoint --pipeline_config_path='/content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/pipeline.config' --model_dir='/content/drive/MyDrive/SSD_mobilenet/fine_tuned_model/frozen_inference_graph.pb' \
 --run_once=True
        # --pipeline_config_path=pipeline_config.pbtxt


python3: can't open file 'model_main.py': [Errno 2] No such file or directory
